***FCIM.FIA - Fundamentals of Artificial Intelligence***

> **Lab 6:** *Natural language Processing and Chat Bots* \\
> **Performed by:** *Bajenov Sevastian*, group *FAF-213* \\
> **Verified by:** Elena Graur, asist. univ.

## Imports and Utils

Create a virtual environment, install all the necessary dependencies so that you can run the notebook using your virtual environment as a kernel.

In [1]:
# pip install -r requirements.txt

## Task 1-2

In this laboratory work the main task was to study, investigate and implement recurrent neural network, in particular Seq2Seq model with LSTM layer in order to use it within the Telegram chatbot. First of all, to construct and train such model it is necessary to construct proper dataset. I decided to use various data about Moldova and its capital, Chisinau, in the form of questions and answers. Initially my questions sounded very similar but with the help of ChatGPT I paraphrased them and made simpler. Finally, the questions and answers were presented in the form of a csv file:

```
Question,Answer
"What is the capital of Moldova?","Chisinau is the capital of Moldova."
"What is the climate in Moldova?","Moldova has a mild and comfortable climate."
```

Summing up, the dataset contains about 160 questions which should be enough to train the model for simple chatbot interaction.

## Task 3-4

The model was implemented and trained using `pytorch` module. Before diving into the code it is essential to clarify all the theoretical concepts regarding recurrent neural networks and their particular cases.

A `recurrent neural network or RNN` is a deep neural network trained on `sequential` or `time series` data to create a machine learning model that can make sequential predictions or conclusions based on sequential inputs. While traditional `deep learning` networks assume that inputs and outputs are independent of each other, the output of recurrent neural networks depend on the prior elements within the sequence. While future events would also be helpful in determining the output of a given sequence, unidirectional recurrent neural networks cannot account for these events in their predictions. 

`Long Short-Term Memory (LSTM)` is a type of artificial recurrent neural network (RNN) architecture. `LSTM` networks introduce memory cells, which have the ability to retain information over long sequences. Each memory cell has three main components: an `input gate`, a `forget gate`, and an `output gate`. These gates help regulate the flow of information in and out of the memory cell. There are also two important components of the `LSTM`: `cell` state and `hidden` state. The former one maintains long-term memory, and stores information over time and the latter captures short-term dependencies, and reflects the current state of the network.

* The `input gate` determines how much of the new input should be stored in the memory cell.

* The `forget gate` decides which information to discard from the memory cell.

* The `output gate` controls how much of the memory cell’s content should be used to compute the hidden state.

The problems from the traditional `RNNs` which are solved by the `LSTM` are `vanishing/exploding gradient` problems. This happens because the gradients of the loss function with respect to earlier layers in a `RNN` can get exponentially smaller or larger respectively as the backpropagation proceeds to update the weights of the model. The gate mechanism of the `LSTM` solves these problems in comparison to the `RNN` in which the memoization happens by replicating the network for consecutive inputs.

Besides `LSTM` I could also use `GRU` architecture for the task of the laboratory work. `GRU` stands for `Gated Recurrent Unit`, that is similar to `LSTM`. However, `GRU` has a simpler architecture than `LSTM`, with fewer parameters, which can make it easier to train and more computationally efficient. The main difference between `GRU` and `LSTM` is the way they handle the memory cell state. In `GRU`, the memory cell state is replaced with a `candidate activation vector`, which is updated using two gates: the `reset gate` and `update gate`. The former determines how much of the previous `hidden state` to forget, while the latter one determines how much of the `candidate activation vector` to incorporate into the new `hidden state`. Although `GRU` has its advantages I decided not to use because it may be more prone to overfitting than `LSTMs`, especially on smaller datasets just like in my case with less than 200 questions and answers.

It is also important to clarify the situation with the `Seq2Seq` models. A `Sequence to Sequence` network, or `Encoder Decoder` network, is a model consisting of two `RNNs` called the `encoder` and `decoder`. The `encoder` reads an input sequence and outputs a single vector, and the `decoder` reads that vector to produce an output sequence. For every input word the encoder outputs a vector and a hidden state, and uses the hidden state for the next input word. The decoder takes the encoder output vector(s) and outputs a sequence of words to create the answer to the question in our situation. With that having been explained it is time to move to the actual implementation of the model in `pytorch`.

First of all, all the parameters for the model have to be initialized. They affect the model performance directly, like for example `INPUT_DIM` and `OUTPUT_DIM`, which represent the size of the vocabulary for input and output of the model. 

Further on, the `Tokenizer` class is designed to perform word analysis and tokenization. It initializes with two dictionaries: `word2idx` for mapping words to unique indices and `idx2word` for reversing this mapping. These dictionaries include predefined tokens for padding (`<PAD>`), start of a sequence (`<SOS>`), end of a sequence (`<EOS>`), and unknown words (`<UNK>`). The `fit` method processes a list of sentences to build a vocabulary, assigning unique indices to all words found in the sentences while updating the dictionaries. The `encode` method converts a given sentence into a list of indices based on the `word2idx` dictionary, using the `<UNK>` index for words not in the vocabulary. Conversely, the `decode` method reconstructs a sentence from a list of indices, excluding special tokens like `<PAD>`, `<SOS>`, and `<EOS>`.

In [ ]:
# do not run this cell

INPUT_DIM = 700
OUTPUT_DIM = 700
HIDDEN_DIM = 256
EMBED_DIM = 64
NUM_LAYERS = 1
LEARNING_RATE = 0.001
EPOCHS = 350
BATCH_SIZE = 32
MAX_LEN = 20


class Tokenizer:
    def __init__(self):
        self.word2idx = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.idx2word = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}

    def fit(self, sentences):
        unique_words = set()
        for sentence in sentences:
            unique_words.update(sentence.split())

        for word in unique_words:
            idx = len(self.word2idx)
            self.word2idx[word] = idx
            self.idx2word[idx] = word

    def encode(self, sentence):
        return [self.word2idx.get(word, self.word2idx["<UNK>"]) for word in sentence.split()]

    def decode(self, indices):
        return " ".join([self.idx2word[idx] for idx in indices if idx not in [0, 1, 2]])

Going further, the `ChatDataset` class is a custom dataset implementation designed for handling question-answer data. It extends pytorch `Dataset` class, which provides the framework for loading and processing data. The class is initialized with a dataset in a tabular format and a tokenizer object to encode text into numerical representations. 

The `__len__` method returns the total number of samples in the dataset, allowing `DataLoader` to iterate through the data. The `__getitem__` method retrieves a single sample, identified by its index, and processes it. Specifically, it extracts the question and answer text from the dataset, encodes them into sequences of numerical indices using the tokenizer, and appends start-of-sequence `<SOS>` and end-of-sequence `<EOS>` tokens. The encoded question and answer are then converted into tensors and returned as a pair (src and trg).

In [ ]:
# do not run this cell

import torch
from torch.utils.data import Dataset


class ChatDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        question = self.data.iloc[idx]["Question"]
        answer = self.data.iloc[idx]["Answer"]

        src = torch.tensor([1] + self.tokenizer.encode(question) + [2])
        trg = torch.tensor([1] + self.tokenizer.encode(answer) + [2])

        return src, trg

Next up, are the `Encoder` and `Decoder` classes are core components of a sequence-to-sequence model.

The `Encoder` processes an input sequence and generates context information that the `Decoder` uses to generate an output sequence. It is initialized with parameters for the `input vocabulary size` (input_dim), `embedding size` (embed_dim), `hidden state size` (hidden_dim), and the `number of LSTM layers` (num_layers).

* `nn.Embedding` layer maps each input token into a dense vector representation (word embedding).
* `nn.LSTM` processes the sequence of embeddings, capturing both past and contextual information. The `LSTM` outputs the hidden and cell states, which encapsulate the encoded input sequence information.
* In the `forward` method, the `Encoder` takes the input sequence, generates its embeddings, and passes them through the `LSTM`. It returns the final `hidden and cell states`, which act as the initial states for the `Decoder`.

The `Decoder` generates the output sequence token by token, using the context provided by the `Encoder`. It is initialized with parameters for the `output vocabulary size` (output_dim), `embedding size` (embed_dim), `hidden state size` (hidden_dim), and the `number of LSTM layers` (num_layers). I decided to use single layer for `LSTM` to avoid overfitting and simplify the model.

* `nn.Embedding` layer maps each input token into a dense vector representation.
* `nn.LSTM` processes these embeddings, conditioned on the `hidden and cell states` from the `Encoder` (or its own states for subsequent tokens).
* `nn.Linear` layer (fully-connected) maps the LSTM's output to the size of the output vocabulary, producing a prediction for the next token.
* In the `forward` method, the `Decoder` takes a single token, the `current hidden state`, and the `cell state`. It embeds the token, processes it through the LSTM, and computes the next token prediction. It returns the prediction along with the updated hidden and cell states, enabling iterative generation of the output sequence.

In [ ]:
# do not run this cell

import torch.nn as nn


class Encoder(nn.Module):

    def __init__(self, input_dim, embed_dim, hidden_dim, num_layers):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_dim, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim,
                            num_layers, batch_first=True)

    def forward(self, x):
        embedded = self.embedding(x)
        _, (hidden, cell) = self.lstm(embedded)
        return hidden, cell


class Decoder(nn.Module):

    def __init__(self, output_dim, embed_dim, hidden_dim, num_layers):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(output_dim, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim,
                            num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, hidden, cell):
        x = x.unsqueeze(1)
        embedded = self.embedding(x)
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        prediction = self.fc(output.squeeze(1))
        return prediction, hidden, cell

Finally, goes the model class `Seq2Seq` which implements actual `sequence-to-sequence` model. It takes an `Encoder` to process the input sequence into context information (hidden and cell states), a `Decoder` to generate the output sequence, and a `device` specifying whether computations should run on a CPU or GPU. 

In the `forward` method a probability distribution over the target vocabulary is produced for each position, stored in a pre-initialized tensor. The next input token is determined either by `teacher forcing` (using the actual target token with a probability defined by `teacher_forcing_ratio`) or by selecting the most probable token predicted by the decoder. This process continues until the entire target sequence is generated. The model is designed to support both training and inference.

In [ ]:
# do not run this cell

import torch
import torch.nn as nn


class Seq2Seq(nn.Module):

    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.fc.out_features
        outputs = torch.zeros(batch_size, trg_len,
                              trg_vocab_size).to(self.device)

        hidden, cell = self.encoder(src)
        input = trg[:, 0]

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[:, t] = output
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            input = trg[:, t] if teacher_force else output.argmax(1)

        return outputs

The following piece of code does not need any explanation while it just initializes the dataset, splits it into training and validation sets, and after that prepares the model. The only clarification which needs to be made relates to the `prepare_batch` method. It is used by each `DataLoader` to transform each batch into source and target batches by reshaping the input and padding the resulting batch sequences. This format makes the data suitable for the model to consume it.

In [ ]:
# do not run this cell

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split


device = torch.device("cpu")

data = pd.read_csv("dataset.csv")
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

tokenizer = Tokenizer()
tokenizer.fit(data["Question"].tolist() + data["Answer"].tolist())

train_dataset = ChatDataset(train_data, tokenizer)
val_dataset = ChatDataset(val_data, tokenizer)


def prepare_batch(batch):
    src_batch, trg_batch = zip(*batch)

    src_batch = pad_sequence(src_batch, batch_first=True, padding_value=0)
    trg_batch = pad_sequence(trg_batch, batch_first=True, padding_value=0)

    return src_batch, trg_batch


train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=prepare_batch)
val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE, collate_fn=prepare_batch)

encoder = Encoder(INPUT_DIM, EMBED_DIM, HIDDEN_DIM, NUM_LAYERS).to(device)
decoder = Decoder(OUTPUT_DIM, EMBED_DIM, HIDDEN_DIM, NUM_LAYERS).to(device)
model = Seq2Seq(encoder, decoder, device).to(device)

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

Once all the necessary classes are initialized it is time to train the model. The `train_model` method takes the `model`, `data loaders` for training and validation, a `loss function` and an `optimizer` as inputs. The process runs for a specified number of epochs having been set at the very beginning.

During the training phase for each batch of `source` and `target` sequences from the training data, the data is moved to the specified device. The optimizer's gradients are reset, and the model's predictions are obtained by passing the source and target sequences through the model. The output predictions and target sequences are reshaped to align with the loss function's expected dimensions, excluding the `<SOS>` token.

In the validation phase for each batch in the validation dataset, the `source` and `target` sequences are passed through the model, and the predictions are compared to the targets to calculate the validation loss. Gradients are not updated during this phase.

At the end of each epoch, the average training and validation losses are computed and printed to monitor the model's performance.

In [ ]:
# do not run this cell

def train_model(model, train_loader, val_loader, criterion, optimizer):
    for epoch in range(EPOCHS):
        model.train()
        train_loss = 0

        for src, trg in train_loader:
            src, trg = src.to(device), trg.to(device)

            optimizer.zero_grad()
            output = model(src, trg)
            output = output[:, 1:].reshape(-1, OUTPUT_DIM)

            trg = trg[:, 1:].reshape(-1)
            loss = criterion(output, trg)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        model.eval()
        val_loss = 0

        with torch.no_grad():
            for src, trg in val_loader:
                src, trg = src.to(device), trg.to(device)

                output = model(src, trg)
                output = output[:, 1:].reshape(-1, OUTPUT_DIM)
                trg = trg[:, 1:].reshape(-1)

                val_loss += criterion(output, trg).item()

        print(f"Epoch {epoch+1}/{EPOCHS}, Train Loss: {train_loss /
              len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}")

Summing up everything said before, the model was trained, the final `training loss` was approximately 0.04-0.05 and the final `validation loss` being between 3 and 4. These results were obtained after reducing the initial number of `LSTM` layers, vocabulary size for input and output, and adjusting the dataset entries. Initial training attempts led to a stronger overfitting and extreme validation loss. So, the final results are the best one and were saved (in the form of the `state_dict` for the model and tokenizer) and used further in the chat bot implementation.

For interested readers the `lstm_training.py` file contains `Tensorflow` implementation of the same model. It can be trained but you should complete the testing process by yourself.

## Task 5-6

In order to setup the Telegram chat bot I interacted with the `Bot Father` to obtain the api key. Then I installed the `pyTelegramBotAPI` module to interact with the bot in python. There are basically to straightforward methods which I implemented. The `send_welcome` method handles `/start` and `/help` commands and outputs welcome message. The `send_answer` method receives user question as input, verifies its contents and length and then sends back the answer using `prepare_answer` method which will be discussed later. Finally, the `bot.infinite_polling()` statement makes the bot listen for the incoming requests for an indefinite period of time.

In [ ]:
# do not run this cell

import telebot
import os
from dotenv import load_dotenv


load_dotenv()
bot = telebot.TeleBot(os.getenv('TOKEN'))


@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    bot.reply_to(
        message,
        "Welcome to this chatbot!\n\n"
        "You can ask any question about Moldova and in particular its capital - Chisinau!\n\n"
        "For example, you can ask:\n"
        "- What is the climate in Moldova?\n"
        "- Where can I watch movies in Chisinau?")


@bot.message_handler(func=lambda message: message.text is not None and len(message.text) > 5 and '/' not in message.text)
def send_answer(message):
    answer = prepare_answer(model, message.text, tokenizer)
    bot.reply_to(message, answer)


bot.infinity_polling()


Coming back to the `prepare_answer` method, it makes use of the already trained model to predict the answers to the questions. First up, the model is redeclared and its state and tokenizer are loaded from the respective files. Then the `prepare_answer` is declared with the `model`, `question` and `tokenizer` as parameters. 

It starts by setting the model to `evaluation mode`. The input question is tokenized and encoded into a sequence of integers with special `<SOS>` and `<EOS>` tokens marking the start and end, respectively. This encoded sequence is converted into a tensor and passed to the model's encoder, which outputs the `initial hidden and cell states`. Decoding begins with the `<SOS>` token, and at each step, the decoder predicts the next token based on the `hidden state`, `cell state`, and the `current input token`. The predicted token is added to the result, and the process continues until the `<EOS>` token is predicted or a maximum sequence length is reached. Finally, the resulting sequence of tokens is decoded back into text using the tokenizer, and the generated response is returned.

In [ ]:
# do not run this cell

import torch
from torch_training import MAX_LEN, INPUT_DIM, OUTPUT_DIM, EMBED_DIM, HIDDEN_DIM, NUM_LAYERS, Tokenizer, Encoder, Decoder, Seq2Seq


device = torch.device("cpu")

tokenizer = torch.load('./models/moldova_tokenizer.pth', weights_only=False)

encoder = Encoder(INPUT_DIM, EMBED_DIM, HIDDEN_DIM, NUM_LAYERS).to(device)
decoder = Decoder(OUTPUT_DIM, EMBED_DIM, HIDDEN_DIM, NUM_LAYERS).to(device)
model = Seq2Seq(encoder, decoder, device).to(device)

model.load_state_dict(torch.load(
    './models/moldova_seq2seq_torch.pth', weights_only=True))


def prepare_answer(model, question, tokenizer):
    model.eval()

    src = torch.tensor([1] + tokenizer.encode(question) +
                       [2]).unsqueeze(0).to(device)
    hidden, cell = model.encoder(src)
    input_token = torch.tensor([1]).to(device)

    result = []

    for _ in range(MAX_LEN):
        output, hidden, cell = model.decoder(input_token, hidden, cell)
        pred_token = output.argmax(1).item()

        if pred_token == 2:
            break

        result.append(pred_token)
        input_token = torch.tensor([pred_token]).to(device)

    return tokenizer.decode(result)

After asking the chat bot several questions I came to the conclusion that it is far from the ideal conditions. Because of the rather small dataset and lack of generalization the model often confuses questions but there are also questions to which the model generates clear and correct answers. Anyway even if the answer does not fit the question, its structure is correct and does not seem to be a random sequence of words. The main problem still remains overfitting and therefore unlike the `CNNs`, the `LSTM` model constructed in this laboratory work might confuse the questions because of it by remembering wrong dependencies between questions and answers.

## Conclusions:

In this laboratory work I learned many important concepts regarding the architecture and implementation of recurrent neural networks. I started from the traditional recurrent networks and then learned the advantages of other approaches, like LSTM and GRU and the difference between them. Further on I applied the theoretical knowledge base to construct the actual LSTM model, for which I prepared suitable dataset. I learned the necessity of word embeddings, encoding and decoding in the process of predicting word sequences. Finally, I fine-tuned the model using different hyperparameters and connected it to the Telegram bot using official api. Although, the chat bot answers were not perfect but the model was trained successfully; obviously, in order to obtain better results it is recommended to prepare a more various dataset with many paraphrased questions without too similar patterns.

## Acknowledgements



In this laboratory work I was assisted by Alexandru Andries and Arteom Kalamaghin from FAF-211. They helped me understand how to construct LSTM model properly, how to enhance my dataset and which tools are most appropriate for building a seq2seq model.

## Bibliography:

1. https://www.ibm.com/topics/recurrent-neural-networks
2. https://medium.com/@rebeen.jaff/what-is-lstm-introduction-to-long-short-term-memory-66bd3855b9ce
3. https://www.baeldung.com/cs/lstm-vanishing-gradient-prevention
4. https://medium.com/@anishnama20/understanding-gated-recurrent-unit-gru-in-deep-learning-2e54923f3e2
5. https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html#:~:text=A%20Sequence%20to%20Sequence%20network%2C%20or%20seq2seq%20network%2C%20or%20Encoder,to%20produce%20an%20output%20sequence.
6. https://medium.com/@rubyabdullah14/building-a-telegram-bot-with-python-a-step-by-step-guide-5ca305bea6c0
7. The whole bunch of videos on recurrent neural networks by StatQuest.